# Compute and compare different radial potentials

In [1]:
using JAC

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")


We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct `Radial.Potential` that (may) contain a name the radial function and the radial grid on which the potential has been defined. The potential function `Zr` typically contains the effective charge $Z(r)$ as seen by an electron, so that the full radial potential is given by $V = -\:Z(r) / r$. This convention has been frequently applied in atomic physics and has been adopted in JAC.


In [2]:
? Radial.Potential

`struct  Radial.Potential`  ... defines a struct for the radial potential which contains all information about its physical content.

```
+ name           ::String            ... A name for the potential.
+ Zr             ::Array{Float64,1}  ... radial potential function Z(r) = - r * V(r) as usual in atomic structure theory.
+ grid           ::RadialGrid        ... radial grid on which the potential is defined.
```

---

`JAC.Radial.Potential()`  ... constructor to define an 'empty' instance of the radial potential.



Perhaps, the simplest choice is the **nuclear** potential which, of course, depends on the nuclear model.
We consider a $^{12}$C carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [3]:
grid     = JAC.Radial.Grid("grid: exponential")
nucModel = Nuclear.Model(7., "Fermi")

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines wit break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


Fermi nuclear model for Z = 7.0 with mass = 14.245, radius R = 2.59656449033488 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.


Then, the nuclear potential is generated by simply calling:


In [4]:
nucPotential = JAC.Nuclear.nuclearPotential(nucModel, grid)

nuclear-potential: Fermi-distributed (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, 0.0040678, 0.0206594, 0.0474903, 0.0799277, 0.112363, 0.139189, 0.155776, 0.165613, 0.189148  …  0.357197, 0.3807, 0.394636, 0.427974, 0.48186, 0.546959, 0.611998, 0.665742, 0.698951, 0.718637]  ...  [7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0  …  7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge $Z = 6.$ at larger $r$-values, i.e. outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nuclues. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the $1s^2 2s^2 2p^2$ ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [5]:
basis     = perform("computation: SCF", [Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = perform("computation: CI",  basis, nucModel, grid, AsfSettings());

shellOccList = SubString{String}["1s^2", "2s^2", "2p^2"]

... in perform('computation: SCF', ...
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_3/2^2 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 7.0 with mass = 14.245, radius R = 2.59656449033488 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -2.45159992e+01    -2.45160029e+01    +1.54138578e-07    
      2      2s_1/2      -6.13000214e+00    -6.13000125e+00    -1.45559923

  1s_1/2::  en [a.u.] = -1.45646721e+01;   self-consistency = 2.176399e-06  [4.480445e-05 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -1.13057740e+00;   self-consistency = 8.935101e-06  [4.480445e-05 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -7.12117554e-01;   self-consistency = 3.565146e-06  [4.926616e-06 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -7.11241670e-01;   self-consistency = 4.250940e-06  [1.874336e-05 for symmetry block kappa = -2]

Iteration 10 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.45646648e+01;   self-consistency = 2.509708e-07  [2.811602e-06 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -1.13057549e+00;   self-consistency = 8.437652e-07  [2.811602e-06 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -7.12120071e-01;   self-consistency = 1.767584e-06  [2.807769e-06 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -7.11243369e-01;   self-consistency = 1.193963e-06  [5.816368e-06 for symmetry block kappa = -2]



From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. We have moreover different choices to generate such an **electronic** potential as seen from ? compute at the REPL:

    •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           core-Hartree potential from the given list of orbitals; cf. JAC.computePotentialCoreHartree. 
           A potential::RadialPotential is returned. 

    •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           Kohn-Sham potential from the given list of orbitals; cf. JAC.computePotentialKohnSham. 
           A potential::RadialPotential is returned.

    •    ("radial potential: Dirac-Fock-Slater", grid::Radial.Grid, level::Level) ... to compute a (radial) 
          Dirac-Fock-Slater potential from the given list of orbitals; this potential is rather simple 
          but includes some undesired self-interaction and exhibits an asymptotic behaviour.         
          Cf. JAC.computePotentialDFS. A potential::RadialPotential is returned.


In [7]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)

coreSubshells = Subshell[1s_1/2, 2s_1/2];    subshells = Subshell[1s_1/2, 2s_1/2, 2p_1/2, 2p_3/2]


DFS (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, -2.47647e-7, -1.27382e-6, -2.92539e-6, -4.91967e-6, -6.91618e-6, -8.56923e-6, -9.59175e-6, -1.01982e-5, -1.16491e-5  …  -2.20035e-5, -2.34523e-5, -2.43117e-5, -2.63682e-5, -2.96951e-5, -3.37187e-5, -3.77429e-5, -4.10712e-5, -4.3129e-5, -4.43494e-5]  ...  [-5.9999, -5.9999, -5.9999, -5.99991, -5.99993, -5.99995, -5.99997, -5.99998, -5.99998, -5.99999  …  -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


All these potentials are represented as *effective charge* $Z(r)$ and can be added to the nuclear potential, for example:

In [8]:
JAC.add(nucPotential, potDFS)

nuclear-potential: Fermi-distributed+nuclear-potential: Fermi-distributed (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, 0.00406755, 0.0206581, 0.0474874, 0.0799228, 0.112356, 0.13918, 0.155766, 0.165602, 0.189136  …  0.357175, 0.380676, 0.394612, 0.427948, 0.481831, 0.546926, 0.61196, 0.665701, 0.698908, 0.718593]  ...  [1.0001, 1.0001, 1.0001, 1.00009, 1.00007, 1.00005, 1.00003, 1.00002, 1.00002, 1.00001  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


What can be seen ... and how to display these potentials also graphically; use `JAC.plot()`